In [8]:
import os
import logging
import pandas as pd
import mysql.connector

In [9]:
MYSQL_USERNAME=os.environ['MYSQL_USERNAME']
MYSQL_PASSWORD=os.environ['MYSQL_PASSWORD']
MYSQL_DATABASE=os.environ['MYSQL_DATABASE']
MYSQL_HOST=os.environ['MYSQL_HOST']

try:
    # Establish connection
    conn = mysql.connector.connect(
        host=MYSQL_HOST,  # Change to your MySQL server
        user=MYSQL_USERNAME,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE  # Optional: specify database
    )

    if conn.is_connected():
        print("Connected to MySQL successfully!")

except mysql.connector.Error as e:
    print("Error:", e)

Connected to MySQL successfully!


In [10]:
# Create a cursor object
cursor = conn.cursor()

In [11]:
def extract(cursor, query):
    # Execute a query
    cursor.execute(query)
    data = cursor.fetchall()
    
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    
    return df

In [12]:
query="SELECT * FROM Store;"
store_df = extract(cursor, query)

In [13]:
store_df

,StoreID,Location,Size,ManagerID
0,1,"123 Main St, New York, NY",Large,1
1,2,"456 Elm St, Los Angeles, CA",Medium,1
2,3,"789 Maple St, Chicago, IL",Small,2
3,4,"101 South St, Miami, FL",Large,6
4,5,"202 North Ave, Seattle, WA",Medium,5
5,6,"303 West Ave, Austin, TX",Small,4
6,7,"404 East Rd, San Francisco, CA",Large,3
7,8,"505 Central Blvd, Dallas, TX",Medium,7
8,9,"606 Park St, Boston, MA",Small,8
9,10,"707 Broadway, Denver, CO",Medium,1


In [14]:
# Close the connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.


In [15]:
import psycopg2
from psycopg2 import sql

In [16]:
PG_USERNAME=os.environ['PG_USERNAME']
PG_PASSWORD=os.environ['PG_PASSWORD']
PG_DATABASE=os.environ['PG_DATABASE']
PG_HOST=os.environ['PG_HOST']

print(PG_HOST, PG_USERNAME, PG_PASSWORD, PG_DATABASE)

172.18.0.2 biruni P@ssw0rd walmart_dwh


In [17]:
# Database connection parameters
db_config = {
    'dbname': PG_DATABASE,
    'user': PG_USERNAME,
    'password': PG_PASSWORD,
    'host': PG_HOST,  # or your database host
    'port': 5432          # default PostgreSQL port
}

In [18]:
# Connect to the database
try:
    conn = psycopg2.connect(**db_config)
    print("Connected to the database!")
except Exception as e:
    print(f"Unable to connect to the database: {e}")


Connected to the database!


In [19]:
# Create a cursor object to interact with the database
cursor = conn.cursor()

In [20]:
query = sql.SQL(f"""
    INSERT INTO dimstore (storebk, location, size, managername)
    VALUES (%s, %s, %s, %s);
""")

In [21]:
def load(cursor, query, data):
    for i, row in data.iterrows():

        try:
            cursor.execute(query, tuple(row))
        except Exception as e:
            conn.rollback()  # Rollback in case of error
            print(f"Error loading store: {e}")

    cursor.execute('commit')
    print("Data Loaded successfully!")

In [23]:
load(cursor, query, store_df)

Data Loaded successfully!


In [24]:
# Close the cursor and connection
cursor.close()
conn.close()
print("Connection closed.")

Connection closed.
